In [ ]:
#|default_exp llm

In [ ]:
#|export
from traitlets import HasTraits, Unicode
from langchain_openai import ChatOpenAI
import os
from langchain.docstore.document import Document
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
#|export
class LLM(HasTraits):

    def __init__(self, filepath='OPENAI_API_KEY'):
        super().__init__()

        with open(filepath, 'r') as file:
            openai_api_key = file.read().strip()
        os.environ['OPENAI_API_KEY'] = openai_api_key
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
llm_model = LLM('OPENAI_API_KEY')
llm_model

<__main__.LLM>

In [ ]:
#| export
class FileModel(LLM):
    # Define a Unicode string trait
    select = Unicode()

    def __init__(self):
        super().__init__()
        #self.embeddings = OpenAIEmbeddings()
        #doc = Document(page_content='Course')
        #self.db = FAISS.from_documents(doc, self.embeddings)

    def save_content_from_upload(values):
        for value in values:
            with open('course_files/' + value['name'], "wb") as fp:
                fp.write(value['content'])

    def load_text(self, text):
        doc = Document(page_content=text)
        db = FAISS.from_documents(doc, self.embeddings)
        self.db.merge_from(db)

    def load_pdf(self, filepath):
        loader = PyPDFLoader(filepath) 
        pages = loader.load_and_split()
        db = FAISS.from_documents(pages, self.embeddings)
        self.db.merge_from(db)

    def load_markdown(filepath):
        loader = UnstructuredMarkdownLoader(filepath, mode="elements") #mode=elements breaks up the text into chunks
        doc = loader.load()
        db = FAISS.from_documents(doc, embeddings)
        self.db.merge_from(db)

    def save_content_from_upload(values):
        for value in values:
            with open(value['name'], "wb") as fp:
                fp.write(value['content'])

In [ ]:
file_model = FileModel()
#file_model.load_pdf("STP 420 spring 2024 course syllabus.pdf") #file as input

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()